In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [2]:
# our API access function. This will be called by the button when it is clicked
def access_api():  
    
    print("Downloading data")
    filters = [
        "areaType=overview"
    ]

    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate",
    }

    api = Cov19API(filters=filters, structure=structure)
    overview = api.get_json()

    filters = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
        "males": "maleCases",
        "females": "femaleCases"
    }

    api = Cov19API(filters=filters, structure=structure)
    agedistribution = api.get_json()

    filters = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
        "date": "date",
        "newAdmissions": "newAdmissions",
        "covidOccupiedMVBeds": "covidOccupiedMVBeds"
    }

    api = Cov19API(filters=filters, structure=structure)
    ventilator = api.get_json()

    filters = [
        'areaType=region',
        'areaName=London'
    ]

    structure = {
        "date": "date",
        "deaths": "newDeaths28DaysByDeathDate",
        "cases": "newCasesBySpecimenDate"
    }

    api = Cov19API(filters=filters, structure=structure)
    london_cases_deaths = api.get_json()

    with open("overview.json", "wt") as OUTF:
        json.dump(overview, OUTF)

    with open("agedistribution.json", "wt") as OUTF:
        json.dump(agedistribution, OUTF)

    with open("ventilator.json", "wt") as OUTF:
        json.dump(ventilator, OUTF)

    with open("london_cases_deaths.json", "wt") as OUTF:
        json.dump(london_cases_deaths, OUTF)
        
    
    print("API call finished succesfully")

    # Ignore the parameter, put code for polling the API here
    print("I'm downloading data from the API...")
    print("...all done.")
    
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api())

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)


API call finished succesfully
I'm downloading data from the API...
...all done.


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [3]:
with open("overview.json", "rt") as INF:
        overview_json=json.load(INF)

datalist = overview_json["data"]

dates = [dictionary["date"] for dictionary in datalist]
dates.sort()

def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

index=pd.date_range(startdate, enddate, freq='D')
overviewdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])


for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['cases', 'hospital', 'deaths']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(overviewdf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            overviewdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
overviewdf.fillna(0.0, inplace=True)
            
overviewdf
print(overviewdf)

overviewdf.to_pickle("overviewdf.pkl")
overviewdf = pd.read_pickle("overviewdf.pkl")

series = wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls = wdg.HBox([series, scale])



def overview_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        overviewdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_overview = wdg.interactive_output(overview_graph, {'gcols': series, 'gscale': scale})

display(controls, output_overview)



2020-01-31 00:00:00  to  2021-07-25 00:00:00
              cases  hospital  deaths
2020-01-31      2.0       0.0     0.0
2020-02-01      0.0       0.0     0.0
2020-02-02      0.0       0.0     0.0
2020-02-03      0.0       0.0     0.0
2020-02-04      0.0       0.0     0.0
...             ...       ...     ...
2021-07-21  44104.0       0.0   193.2
2021-07-22  39906.0       0.0   193.3
2021-07-23  36389.0       0.0   193.3
2021-07-24  31795.0       0.0     0.0
2021-07-25  29173.0       0.0     0.0

[542 rows x 3 columns]


Output()